<a href="https://colab.research.google.com/github/goransavich/cnn-xgboost/blob/main/cnn-xgboost-final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [159]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot
np.random.seed(1337)  # for reproducibility
#from keras.utils import np_utils
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Flatten, Input, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
#xgboost
import xgboost as xgb
import pandas as pd
from sklearn import metrics

In [160]:
from google.colab import files
uploaded = files.upload()
import io
import pandas as pd

train_name = list(uploaded.keys())[0]

train_df = pd.read_csv(io.BytesIO(uploaded[train_name]))

Saving Coffee_TRAIN.csv to Coffee_TRAIN (9).csv


In [161]:
uploaded = files.upload()

test_name = list(uploaded.keys())[0]

test_df = pd.read_csv(io.BytesIO(uploaded[test_name]))

Saving Coffee_TEST.csv to Coffee_TEST (9).csv


In [162]:
train_data = np.array(train_df)
test_data = np.array(test_df)

In [163]:
x_train = train_data[:, :-1]
x_test = test_data[:, :-1]
y_train = train_data[:, -1:]
y_test = test_data[:, -1:]

classes_in_dataset = len(np.unique(np.concatenate((y_train, y_test), axis=0)))


In [164]:
x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train,test_size = 0.2,random_state = 42)

In [165]:
if len(x_train.shape) == 2:  # if univariate
        # add a dimension to make it multivariate with one dimension 
        x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
input_shape = x_train.shape[1:]

print(input_shape)


print(classes_in_dataset)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_validate.shape)
print(y_validate.shape)

(286, 1)
2
(22, 286, 1)
(22, 1)
(28, 286, 1)
(28, 1)
(6, 286)
(6, 1)


In [166]:
inputs = input_shape
input_layer = Input(inputs)
conv2d1 = Convolution1D(filters=64, kernel_size=1, padding='valid')(input_layer)
activation1 = Activation('relu')(conv2d1)
maxpooling1 = MaxPooling1D(pool_size=2, strides=1, padding='valid')(activation1)
#dropout1 = Dropout(0.5)(maxpooling1)

conv2d2 = Convolution1D(filters=62, kernel_size=1, padding='valid')(maxpooling1)
activation2 = Activation('relu')(conv2d2)
maxpooling2 = MaxPooling1D(pool_size=2, strides=1, padding='valid')(activation2)
dropout2 = Dropout(0.5)(maxpooling2)

flatten = Flatten()(maxpooling2)

output_for_xgboost = Model(inputs=input_layer, outputs=flatten)

######### XGBOOST #############
x_train_xg = output_for_xgboost.predict(x_train)
x_test_xg = output_for_xgboost.predict(x_test)

train_x = x_train_xg
test_x = x_test_xg
#print(x_train_xg.shape)
#print(x_test_xg.shape)

dataset = xgb.DMatrix(train_x, label=y_train)
watchlist = [(dataset, 'train')]
params = {'max_depth':7, 'eta':0.1, 'silent':1, 'num_class': classes_in_dataset,'objective':'multi:softmax' } 
model_xg = xgb.train(params, dataset, num_boost_round=150, evals=watchlist)


test_x = xgb.DMatrix(test_x)
result = model_xg.predict(test_x)


[0]	train-merror:0
[1]	train-merror:0
[2]	train-merror:0
[3]	train-merror:0
[4]	train-merror:0
[5]	train-merror:0
[6]	train-merror:0
[7]	train-merror:0
[8]	train-merror:0
[9]	train-merror:0
[10]	train-merror:0
[11]	train-merror:0
[12]	train-merror:0
[13]	train-merror:0
[14]	train-merror:0
[15]	train-merror:0
[16]	train-merror:0
[17]	train-merror:0
[18]	train-merror:0
[19]	train-merror:0
[20]	train-merror:0
[21]	train-merror:0
[22]	train-merror:0
[23]	train-merror:0
[24]	train-merror:0
[25]	train-merror:0
[26]	train-merror:0
[27]	train-merror:0
[28]	train-merror:0
[29]	train-merror:0
[30]	train-merror:0
[31]	train-merror:0
[32]	train-merror:0
[33]	train-merror:0
[34]	train-merror:0
[35]	train-merror:0
[36]	train-merror:0
[37]	train-merror:0
[38]	train-merror:0
[39]	train-merror:0
[40]	train-merror:0
[41]	train-merror:0
[42]	train-merror:0
[43]	train-merror:0
[44]	train-merror:0
[45]	train-merror:0
[46]	train-merror:0
[47]	train-merror:0
[48]	train-merror:0
[49]	train-merror:0
[50]	train

In [167]:

from sklearn.metrics import precision_score, recall_score, accuracy_score

#preds = model.predict(d_test)
best_preds = np.asarray([np.argmax(line) for line in result])

print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

Precision = 0.26785714285714285
Recall = 0.5
Accuracy = 0.5357142857142857


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
